# AWS Lambda Deployment with Hydraa FaaS Manager

This notebook demonstrates lambda function deployment (via zip) using the Hydraa FaaS Manager.

## Prerequisites
- AWS credentials configured
- Hydraa FaaS Manager installed
- Lambda function code prepared

In [12]:
from src.hydraa_faas.faas_manager.manager import FaasManager
from hydraa.services import ServiceManager
from hydraa import proxy, AWS, Task
import os
from dotenv import load_dotenv
import shutil

## Setup AWS Credentials

Make sure your AWS credentials are set as environment variables:

In [13]:
# load environment variables from .env file containing your region and keys
load_dotenv()

# verify aws credentials are set
required_vars = ['ACCESS_KEY_ID', 'ACCESS_KEY_SECRET', 'AWS_REGION']
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    raise ValueError(f"missing environment variables: {missing_vars}. please set them in your .env file.")

## Initialize FaaS Manager

Create the provider manager and FaaS manager:

In [14]:
# initialize provider manager
provider_mgr = proxy([AWS])

# create faas manager
faas_mgr = FaasManager(
    provider_mgr, 
    asynchronous=True,
    auto_terminate=True
)

# start service manager
try:
    service_mgr = ServiceManager([faas_mgr])
    service_mgr.start_services()
except FileExistsError as e:
    # handle existing sandbox by cleaning up and restart
    sandbox_path = str(e).split("'")[1]
    shutil.rmtree(sandbox_path)
    service_mgr = ServiceManager([faas_mgr])
    service_mgr.start_services()

login to: ['aws'] succeed
starting faas manager with [1] providers
faas manager started with providers: ['aws']


## Create Lambda Function Tasks

Create lambda test function project directories by running `python3 lambda_functions/create_functions.py` and then define multiple lambda functions for deployment:

In [15]:
# create 10 lambda function tasks using 'real' project directories
tasks = []

for i in range(10):
    task = Task()
    
    # lambda function example configuration
    task.source_path = f'./lambda_functions/function_{i}'
    task.handler_name = 'main.handler'
    task.runtime = 'python3.9'
    task.memory = 256 + (i * 64)  # memory in MB (128-10240)
    task.timeout = 30  # timeout in seconds
    task.provider = AWS
    
    # environment variables that our example functions expect
    task.env_var = [f'FUNCTION_ID={i}', 'ENV=production']
    
    tasks.append(task)

## Deploy Lambda Functions

Submit all tasks for deployment:

In [16]:
# submit all tasks for deployment
faas_mgr.submit(tasks)

10 function(s) submitted for deployment


In [17]:
# wait for all functions to deploy
results = []
deployed_functions = []

for i, task in enumerate(tasks):
    try:
        result = task.result(timeout=300)  # 5 minute timeout per function
        results.append(result)
        deployed_functions.append(task)
    except Exception as e:
        results.append(f"failed: {e}")

## Invoke Lambda Functions

Test the deployed functions:

In [18]:
# invoke the deployed functions
successful_invocations = 0
invocation_results = []

for i, task in enumerate(deployed_functions[:5]):  # test first 5 deployed functions
    try:
        response = faas_mgr.invoke(
            task.name, 
            {
                'message': f'Hello from function {i}!', 
                'iteration': i,
                'test_data': [1, 2, 3, 4, 5]
            }
        )
        
        invocation_results.append({
            'function': task.name,
            'status': response['StatusCode'],
            'success': True
        })
        successful_invocations += 1
            
    except Exception as e:
        invocation_results.append({
            'function': task.name,
            'error': str(e),
            'success': False
        })

print(invocation_results)

[{'function': 'hydraa-function-0', 'status': 200, 'success': True}]


## Check Function Status

Get information about deployed functions:

In [19]:
# list all deployed functions
functions = faas_mgr.list_functions()
function_count = sum(len(func_list) for func_list in functions.values() if isinstance(func_list, list))

In [20]:
# check provider status
status = faas_mgr.get_provider_status()

# display summary
for provider, provider_status in status.items():
    active_status = "Active" if provider_status['active'] else "Inactive"
    function_count = provider_status['functions_count']
    run_id = provider_status['run_id']

## Display Results Summary

Show the results of our Lambda deployment and testing:

In [21]:
# display deployment summary
deployment_summary = {
    'total_tasks': len(tasks),
    'successful_deployments': len(deployed_functions),
    'failed_deployments': len(tasks) - len(deployed_functions),
    'successful_invocations': successful_invocations,
    'total_functions_in_account': function_count
}

# show key metrics
for key, value in deployment_summary.items():
    formatted_key = key.replace('_', ' ').title()

print(deployment_summary)

# get performance metrics
performance_metrics = faas_mgr.get_performance_metrics()
print(f"\nperformance metrics: {performance_metrics}")

# clean up resources
service_mgr.shutdown_services()

{'total_tasks': 10, 'successful_deployments': 1, 'failed_deployments': 9, 'successful_invocations': 1, 'total_functions_in_account': 1}

performance metrics: {'message': 'no deployment metrics available'}
shutting down all faas providers
terminating provider: aws
faas manager shutdown complete


## Summary

This notebook demonstrated:

1. **FaaS Manager Setup** - simple initialization with optimized performance
2. **Function Deployment** - deploying multiple lambda functions from source code with parallel processing
3. **Function Testing** - invoking deployed functions with test payloads
4. **Status Monitoring** - checking provider and function status
5. **Performance Metrics** - viewing deployment analytics
6. **Resource Cleanup** - proper shutdown and cleanup of resources